# Weather Records
Download weather data and build attributes
- historic weather records provided by the German Weather Institution
- Data Preparation

Source:
- [Deutscher Wetter Dienst (DWD)](https://www.dwd.de/DE/leistungen/cdc/cdc_ueberblick-klimadaten.html)

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import requests
import os
import re
from datetime import datetime
from bs4 import BeautifulSoup
import urllib.request
import zipfile

import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio
import rioxarray
import xarray as xr

import dask
from dask import compute, delayed
from dask.distributed import Client

from scipy import interpolate

import hvplot.pandas
from bokeh.resources import INLINE
import bokeh.io
bokeh.io.output_notebook(INLINE)

xr.set_options(display_style="html")
pd.set_option('display.max_columns', None)

ROOT_PATH = "./data/WeatherData"

## Download weather records
Scrape ZIP archives and save on local disk

In [3]:
# HISTORIC WEATHER DATA
ROOT_URL = "https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/daily/kl/historical/"

dwd_links = list()

res = requests.request(method="GET", url=ROOT_URL)
soup = BeautifulSoup(res.text, "html.parser")
for link in soup.find("pre").find_all("a", href=True):
    if "20211231" in link["href"]:
        dwd_links.append(link["href"])

for item in dwd_links[:]:
    if item[:-4] not in os.listdir(ROOT_PATH):
        print(item)

        urllib.request.urlretrieve(f"{ROOT_URL}{item}", f"{ROOT_PATH}/{item}")
        with zipfile.ZipFile(f"{ROOT_PATH}/{item}", 'r') as zip_ref:
            if not os.path.exists(f"{ROOT_PATH}/{item[:-4]}"):
                os.makedirs(f"{ROOT_PATH}/{item[:-4]}")
            zip_ref.extractall(f"{ROOT_PATH}/{item[:-4]}")
        os.remove(f"{ROOT_PATH}/{item}")

# RECENT WEATHER DATA
RECENT_URL = "https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/daily/kl/recent/"

dwd_links = list()

res = requests.request(method="GET", url=RECENT_URL)
soup = BeautifulSoup(res.text, "html.parser")
for link in soup.find("pre").find_all("a", href=True):
    if "tageswerte_KL" in link["href"]:
        dwd_links.append(link["href"])

for item in dwd_links[:]:
    if item[:-4] not in os.listdir(ROOT_PATH):
        print(item)

        urllib.request.urlretrieve(f"{RECENT_URL}{item}", f"{ROOT_PATH}/{item}")
        with zipfile.ZipFile(f"{ROOT_PATH}/{item}", 'r') as zip_ref:
            if not os.path.exists(f"{ROOT_PATH}/{item[:-4]}"):
                os.makedirs(f"{ROOT_PATH}/{item[:-4]}")
            zip_ref.extractall(f"{ROOT_PATH}/{item[:-4]}")
        os.remove(f"{ROOT_PATH}/{item}")

In [5]:
historical_weather_files = list()
weather_stations = list()
missing_records_files = list()

re_daily_data = re.compile(r"produkt\_klima\_tag\_\d{8}\_\d{8}_\d{5}\.txt")
re_geo_loc = re.compile(r"Metadaten\_Geographie_\d{5}\.txt")
re_missing_records = re.compile(r"Metadaten\_Fehlwerte_\d{5}\_\d{8}\_\d{8}\.txt")

for _dir in os.listdir(ROOT_PATH):
    if os.path.isdir(f"{ROOT_PATH}/{_dir}"):
        for file in os.listdir(f"{ROOT_PATH}/{_dir}"):
            if re_daily_data.match(file):
                historical_weather_files.append(f"{ROOT_PATH}/{_dir}/{file}")
            if re_geo_loc.match(file):
                weather_stations.append(f"{ROOT_PATH}/{_dir}/{file}")
            if re_missing_records.match(file):
                missing_records_files.append(f"{ROOT_PATH}/{_dir}/{file}")

print(f"weather files found: {len(historical_weather_files)} for {len(weather_stations)} weather stations")

weather files found: 1144 for 1144 weather stations


## Historic Weather Data to DataFrame

[Reference](https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/daily/kl/historical/DESCRIPTION_obsgermany_climate_daily_kl_historical_en.pdf)

| STATIONS_ID |Stationsidentifikationsnummer|Note|
|-------------| --- | --- |
| MESS_DATUM  |yyyymmdd||
| FX          |daily maximum of wind gust [m/s]||
| FM          |daily mean of wind speed [m/s]||
| QN_4        |quality level of next columns||
| RSK         |daily precipitation height [mm]||
| RSKF        |precipitation form|0=no precipitation;1=only rain;4=unknown form of recorded precipitation;6=only rain;7=only snow;8=rain and snow;9=error|
| SDK         |daily sunshine duration [h]||
| SHK_TAG     |daily snow depth [cm]||
| NM          |daily mean of cloud cover [1/8]||
| VPM         |daily mean of vapor pressure [hPa]||
| PM          |daily mean of pressure [hPa]||
| TMK         |daily mean of temperature [C]||
| UPM         |daily mean of relative humidity [%]||
| TK          |daily maximum of temperature at 2m height [C]||
| TNK         |daily minimum of temperature at 2m height [C]||
| TGK         |daily minimum of air temperature at 5cm above ground [C]||
| eor         |End of data record||




In [197]:
data_frames = list()

for file in historical_weather_files:
    data_frames.append(pd.read_csv(file, sep=";", encoding="utf-8"))

df = pd.concat(data_frames, ignore_index=True)
df.shape

(10013915, 19)

In [198]:
missing_data_frames = list()

for file in missing_records_files:
    missing_data_frames.append(pd.read_csv(file, sep=";", encoding="ANSI"))

df_missings = pd.concat(missing_data_frames, ignore_index=True)
df_missings.shape

(175136, 9)

In [199]:
EARLIEST_DATE = '2013-8-31'

### Identify Missing Values
Missing data for single days will be filled at a later stage

In [200]:
df_missings = df_missings.drop(columns=[col for col in df_missings.columns if col.strip() == ""])
df_missings = df_missings.loc[~df_missings.Stations_ID.str.contains("generiert:")]
df_missings['Von_Datum'] = pd.to_datetime(df_missings['Von_Datum'], format='%d.%m.%Y')
df_missings['Bis_Datum'] = pd.to_datetime(df_missings['Bis_Datum'], format='%d.%m.%Y')

df_missings = df_missings.loc[df_missings["Bis_Datum"] > EARLIEST_DATE]
df_missings['Bis_Datum'] = pd.to_datetime(df_missings['Bis_Datum'], format='%d.%m.%Y')

df_missings["date"] = df_missings.apply(lambda x: pd.date_range(start=x["Von_Datum"], end=x["Bis_Datum"]), axis=1)

df_missings = df_missings.explode("date")

df_missings = df_missings.drop(columns=["Stations_Name", "Parameter", "Von_Datum", "Bis_Datum", "Anzahl_Fehlwerte", "Beschreibung", "eor"])
df_missings = df_missings.rename(columns={"Stations_ID": "station_id"})

### Filter Data

In [203]:
# transoform datetime
df['MESS_DATUM'] = pd.to_datetime(df['MESS_DATUM'], format='%Y%m%d')
# filter min date
df = df.loc[df['MESS_DATUM'] > EARLIEST_DATE]

# drop duplicate rows
df = df.drop_duplicates()

# replace -999 with NaN # DOCS
for col in df.columns:
    df[col] = df[col].replace({-999: np.NaN})
    # strip whitespaces from column names
    df.rename(columns={col: col.strip()}, inplace=True)

col_map = {
    "STATIONS_ID": "station_id",
    "MESS_DATUM": "date",
    "QN_3": "quality_level3",
    "FX": "wind_max",
    "FM": "wind_mean",
    "QN_4": "quality_level4",
    "RSK": "rain_mm",
    "RSKF": "rain_form_code",
    "SDK": "sun_h",
    "SHK_TAG": "snow_cm",
    "NM": "cloud_cover_mean",
    "VPM": "vapor_pressure_mean",
    "PM": "pressure_mean",
    "TMK" : "temp_mean",
    "UPM": "humidity",
    "TXK": "temp_max",
    "TNK": "temp_min",
    "TGK": "temp_min_ground_level",
    "eor": "eor"
}

df.rename(columns=col_map, inplace=True)

df = df.append(df_missings, ignore_index=True)
df = df.loc[df['date'] > EARLIEST_DATE]

df.head()

,station_id,date,quality_level3,wind_max,wind_mean,quality_level4,rain_mm,rain_form_code,sun_h,snow_cm,cloud_cover_mean,vapor_pressure_mean,pressure_mean,temp_mean,humidity,temp_max,temp_min,temp_min_ground_level,eor
0,11,2020-07-13,10.0,8.6,2.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,eor
1,11,2020-07-14,10.0,8.4,1.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,eor
2,11,2020-07-15,10.0,9.5,1.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,eor
3,11,2020-07-16,10.0,6.5,1.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,eor
4,11,2020-07-17,10.0,7.1,2.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,eor
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10013910,19172,2022-01-09,NaN,NaN,NaN,1.0,3.2,4.0,NaN,NaN,NaN,7.1,NaN,2.6,96.38,4.7,1.2,1.0,eor
10013911,19172,2022-01-10,NaN,NaN,NaN,1.0,0.0,0.0,NaN,NaN,NaN,7.1,NaN,2.7,95.54,4.0,1.7,0.6,eor
10013912,19172,2022-01-11,NaN,NaN,NaN,1.0,0.5,4.0,NaN,NaN,NaN,6.9,NaN,2.7,92.29,4.1,1.5,1.7,eor
10013913,19172,2022-01-12,NaN,NaN,NaN,1.0,0.0,4.0,NaN,NaN,NaN,8.2,NaN,4.3,98.42,6.2,2.3,2.3,eor


### Weather Stations
Weather stations do store information about their lat, long and alt

In [6]:
ws_data_frames = list()

# read data
for file in weather_stations:
    ws_data_frames.append(pd.read_csv(file, sep=";", encoding="latin-1"))

# concatenate dataframess
df_weather_stations = pd.concat(ws_data_frames, ignore_index=True)
# parse dates
df_weather_stations['von_datum'] = pd.to_datetime(df_weather_stations['von_datum'], format='%Y%m%d')
df_weather_stations['bis_datum'] = df_weather_stations['bis_datum'].apply(lambda i: "20211231" if str(i).strip() == "" else i)
df_weather_stations['bis_datum'] = pd.to_datetime(df_weather_stations['bis_datum'], format='%Y%m%d')
# rename cols
df_weather_stations.rename(columns={"Stations_id": "station_id", "Stationshoehe": "alt", "Geogr.Breite": "lat", "Geogr.Laenge": "long", "Stationsname": "station_name"}, inplace=True)

df_weather_stations.shape

(4550, 7)

In [7]:
from pyproj import Proj, transform

inProj = Proj(init='epsg:4326')
outProj = Proj(init='epsg:32632')

def transform_epsg_4326_32632(x1, y1)->pd.Series:
    """EPSG 32632 from 4326"""
    x2, y2 = transform(inProj, outProj, x1, y1)
    return pd.Series([x2, y2])

df_weather_stations[["x", "y"]] = df_weather_stations.apply(lambda x: transform_epsg_4326_32632(x["long"], x["lat"]), axis=1)
df_weather_stations.head()

#### Weather Stations to Weather Records

In [213]:
df[["alt", "y", "x", "station_name"]] = np.NaN

for _index, weather_station in df_weather_stations.iterrows():
    df.loc[(df["station_id"] == weather_station["station_id"]) & (df["date"] >= weather_station["von_datum"]) & (df["date"] <= weather_station["bis_datum"]), ["alt", "y", "x", "station_name"]] = weather_station[["alt", "y", "x", "station_name"]].values

In [214]:
df = df.sort_values(by="date", ascending=True)

In [216]:
# dump data
df.to_csv(f"{ROOT_PATH}/historic_weather_data.csv", encoding="utf-8", index=False)

In [3]:
df = pd.read_csv(f"{ROOT_PATH}/historic_weather_data.csv", encoding="utf-8", index_col=0)
df.head()

,station_id,date,quality_level3,wind_max,wind_mean,quality_level4,rain_mm,rain_form_code,sun_h,snow_cm,cloud_cover_mean,vapor_pressure_mean,pressure_mean,temp_mean,humidity,temp_max,temp_min,temp_min_ground_level,eor,alt,y,x,station_name
2032074,1550,2013-09-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1888006,760,2013-09-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1993201,1327,2013-09-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1351543,6097,2013-09-01,10.0,20.8,11.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,eor,4.0,6.041883e+06,726855.209237,Darßer Ort (SWN)
1731720,232,2013-09-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
945071,4024,2022-01-13,1.0,19.0,10.1,1.0,0.0,6.0,0.0,0.0,7.9,8.6,1023.82,5.8,93.17,6.6,5.0,4.7,eor,NaN,NaN,NaN,NaN
957482,4063,2022-01-13,NaN,NaN,NaN,1.0,0.0,0.0,NaN,NaN,NaN,7.2,NaN,2.8,96.17,3.5,2.0,1.9,eor,NaN,NaN,NaN,NaN
1342249,5941,2022-01-13,NaN,NaN,NaN,1.0,0.0,0.0,NaN,30.0,NaN,2.7,NaN,-8.8,83.83,1.0,-15.0,-20.4,eor,NaN,NaN,NaN,NaN
102605,368,2022-01-13,NaN,NaN,NaN,1.0,0.0,0.0,NaN,0.0,8.0,7.5,1026.72,3.1,98.46,NaN,NaN,NaN,eor,NaN,NaN,NaN,NaN


In [4]:
df_station_loc = df.loc[~df.x.isna()].groupby("station_id").agg({"x": "mean", "y": "mean", "alt": "mean", "station_name": "first"}).reset_index()
df_station_loc.head()

,station_id,x,y,alt,station_name
0,11,464213.322860,5.313488e+06,680.0,Donaueschingen (Landeplatz)
1,44,448716.677030,5.865157e+06,43.5,Großenkneten
2,71,498395.462077,5.340264e+06,759.0,Albstadt-Badkap
3,73,798511.385603,5.392682e+06,340.0,Aldersbach-Kriestorf
4,78,426160.474464,5.815573e+06,65.0,Alfhausen
...,...,...,...,...,...
579,15911,880095.292942,5.678869e+06,278.0,Großpostwitz-Denkwitz
580,15976,443378.239257,5.533692e+06,216.0,Ober-Olm/Bellem
581,15978,417593.923061,5.484152e+06,316.0,Sembach
582,19171,556058.776371,5.984283e+06,13.0,Hasenkrug-Hardebek


In [5]:
df_station_loc = df.loc[~df.x.isna()].groupby("station_id").agg({"x": "mean", "y": "mean", "alt": "mean", "station_name": "first"}).reset_index()
df = df.merge(df_station_loc, on="station_id", how="left", suffixes=('', '_fill'))
df["x"] = df["x"].fillna(df["x_fill"])
df["y"] = df["y"].fillna(df["y_fill"])
df["alt"] = df["alt"].fillna(df["alt_fill"])
df["station_name"] = df["y"].fillna(df["station_name_fill"])

df = df.drop(columns=["x_fill", "y_fill", "alt_fill", "station_name_fill"])

# remove station id 4485 as no location data is available
df = df.loc[~df.x.isna()]
df

,station_id,date,quality_level3,wind_max,wind_mean,quality_level4,rain_mm,rain_form_code,sun_h,snow_cm,cloud_cover_mean,vapor_pressure_mean,pressure_mean,temp_mean,humidity,temp_max,temp_min,temp_min_ground_level,eor,alt,y,x,station_name
0,1550,2013-09-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,719.250000,5.260901e+06,655351.086731,5.260901e+06
1,760,2013-09-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,82.000000,5.913056e+06,562785.629373,5.913056e+06
2,1327,2013-09-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,147.000000,5.620116e+06,344009.734910,5.620116e+06
3,6097,2013-09-01,10.0,20.8,11.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,eor,4.000000,6.041883e+06,726855.209237,6.041883e+06
4,232,2013-09-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,461.530000,5.365393e+06,643644.349766,5.365393e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2439428,4024,2022-01-13,1.0,19.0,10.1,1.0,0.0,6.0,0.0,0.0,7.9,8.6,1023.82,5.8,93.17,6.6,5.0,4.7,eor,39.500000,6.033297e+06,790808.471097,6.033297e+06
2439429,4063,2022-01-13,NaN,NaN,NaN,1.0,0.0,0.0,NaN,NaN,NaN,7.2,NaN,2.8,96.17,3.5,2.0,1.9,eor,40.705486,5.810743e+06,471998.822206,5.810743e+06
2439430,5941,2022-01-13,NaN,NaN,NaN,1.0,0.0,0.0,NaN,30.0,NaN,2.7,NaN,-8.8,83.83,1.0,-15.0,-20.4,eor,685.500000,5.286057e+06,760436.231279,5.286057e+06
2439431,368,2022-01-13,NaN,NaN,NaN,1.0,0.0,0.0,NaN,0.0,8.0,7.5,1026.72,3.1,98.46,NaN,NaN,NaN,eor,68.460000,5.852114e+06,562327.616829,5.852114e+06


## Records to Raster Data

In [6]:
weather_columns = ["rain_mm", "rain_form_code", "sun_h", "snow_cm", "cloud_cover_mean", "vapor_pressure_mean", "pressure_mean", "temp_mean", "humidity", "temp_max", "temp_min", "temp_min_ground_level"]

In [8]:
df = df.drop_duplicates(["x", "y", "date"])
df_index = df.set_index(["x", "y", "date"])
ds = df_index.to_xarray()

In [9]:
ds

<xarray.Dataset>
Dimensions:                (x: 747, y: 739, date: 3057)
Coordinates:
  * x                      (x) float64 2.903e+05 2.97e+05 ... 9.159e+05
  * y                      (y) float64 5.25e+06 5.254e+06 ... 6.096e+06
  * date                   (date) object '2013-09-01' ... '2022-01-13'
Data variables: (12/20)
    station_id             (x, y, date) float64 nan nan nan nan ... nan nan nan
    quality_level3         (x, y, date) float64 nan nan nan nan ... nan nan nan
    wind_max               (x, y, date) float64 nan nan nan nan ... nan nan nan
    wind_mean              (x, y, date) float64 nan nan nan nan ... nan nan nan
    quality_level4         (x, y, date) float64 nan nan nan nan ... nan nan nan
    rain_mm                (x, y, date) float64 nan nan nan nan ... nan nan nan
    ...                     ...
    temp_max               (x, y, date) float64 nan nan nan nan ... nan nan nan
    temp_min               (x, y, date) float64 nan nan nan nan ... nan nan nan
    temp_min_ground_level  (x, y, date) float64 nan nan nan nan ... nan nan nan
    eor                    (x, y, date) object nan nan nan nan ... nan nan nan
    alt                    (x, y, date) float64 nan nan nan nan ... nan nan nan
    station_name           (x, y, date) float64 nan nan nan nan ... nan nan nan

## Interpolate Missing Values

In [ ]:
#### Dusk Client for parallel computing
client = Client(n_workers=32)

In [60]:
@dask.delayed
def interpolate_missing_data(data: np.ndarray, mask: np.ndarray, method: str='nearest', fill_value: int=0):
    """
    :param data: a 2D image
    :param mask: a 2D boolean image, True indicates missing values
    :param method: interpolation method, one of
        'nearest', 'linear', 'cubic'.
    :param fill_value: which value to use for filling up data outside the
        convex hull of known pixel values.
        Default is 0, Has no effect for 'nearest'.
    :return: the image with missing values interpolated
    """

    h, w = data.shape[:2]
    xx, yy = np.meshgrid(np.arange(w), np.arange(h))

    known_x = xx[~mask]
    known_y = yy[~mask]
    known_v = data[~mask]
    missing_x = xx[mask]
    missing_y = yy[mask]

    interp_values = interpolate.griddata(
        (known_x, known_y), known_v, (missing_x, missing_y),
        method=method, fill_value=fill_value
    )

    interp_data = data.copy()
    interp_data[missing_y, missing_x] = interp_values

    return interp_data

In [75]:
# linear interpolation
for col in weather_columns:
    delayed_results = [delayed(interpolate_missing_data)(ds[col].sel({"date": date}).values, np.isnan(ds[col].sel({"date": date}).values), "nearest", np.nanmedian(ds[col].sel({"date": date}).values)) for date in ds["date"].values]
    interpolated_values = compute(*delayed_results)
    try:
        ds.update({col:(('date', 'x', 'y'), np.array(interpolated_values))})
    except ValueError:
        ds.update({col:(('date', 'y', 'x'), np.array(interpolated_values))})

ds

<xarray.Dataset>
Dimensions:                (x: 747, y: 739, date: 3057)
Coordinates:
  * x                      (x) float64 2.903e+05 2.97e+05 ... 9.159e+05
  * y                      (y) float64 5.25e+06 5.254e+06 ... 6.096e+06
  * date                   (date) object '2013-09-01' ... '2022-01-13'
Data variables: (12/20)
    station_id             (x, y, date) float64 nan nan nan nan ... nan nan nan
    quality_level3         (x, y, date) float64 nan nan nan nan ... nan nan nan
    wind_max               (date, x, y) float64 10.6 10.6 10.6 ... 7.9 7.9 7.9
    wind_mean              (date, x, y) float64 3.7 3.7 3.7 3.7 ... 3.2 3.2 3.2
    quality_level4         (x, y, date) float64 nan nan nan nan ... nan nan nan
    rain_mm                (date, x, y) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    ...                     ...
    temp_max               (date, x, y) float64 18.0 18.0 18.0 ... 4.0 4.0 4.0
    temp_min               (date, x, y) float64 9.2 9.2 9.2 ... -1.5 -1.5 -1.5
    temp_min_ground_level  (date, x, y) float64 7.35 7.35 7.35 ... -2.6 -2.6
    eor                    (x, y, date) object nan nan nan nan ... nan nan nan
    alt                    (x, y, date) float64 nan nan nan nan ... nan nan nan
    station_name           (x, y, date) float64 nan nan nan nan ... nan nan nan

In [124]:
var_names = list(ds)
dates = list(ds["date"].values)

df_interpolated = pd.DataFrame(data=[], columns=var_names)

unique_stations = df.groupby(["station_id", "x", "y"]).size().reset_index()
for _i, station in unique_stations.iterrows():
    tmp_df = pd.DataFrame(ds.sel(x=station["x"], y=station["y"]).to_array().values.T, columns=var_names)
    tmp_df["date"] = dates
    tmp_df["station_id"] = station["station_id"]
    tmp_df["x"] = station["x"]
    tmp_df["y"] = station["y"]
    df_interpolated = df_interpolated.append(tmp_df, ignore_index=True)
    del tmp_df

df_interpolated.head()

,station_id,quality_level3,wind_max,wind_mean,quality_level4,rain_mm,rain_form_code,sun_h,snow_cm,cloud_cover_mean,vapor_pressure_mean,pressure_mean,temp_mean,humidity,temp_max,temp_min,temp_min_ground_level,eor,alt,station_name,date,x,y
0,11.0,NaN,8.173478,2.038094,NaN,0.08561,0.0,4.654754,0.0,6.796674,12.2102,913.956126,14.419308,74.668488,19.478324,8.659381,7.84827,NaN,NaN,NaN,2013-09-01,464213.322860,5.313488e+06
1,11.0,NaN,9.092413,3.091648,NaN,0.0,0.0,8.54286,0.0,3.454191,8.483979,915.470133,13.26447,61.81137,21.801163,6.271705,4.308915,NaN,NaN,NaN,2013-09-02,464213.322860,5.313488e+06
2,11.0,NaN,7.058365,2.097729,NaN,0.0,0.0,11.991982,0.0,0.581327,11.820947,918.033514,16.294171,66.673953,26.441712,5.558288,4.209836,NaN,NaN,NaN,2013-09-03,464213.322860,5.313488e+06
3,11.0,NaN,6.929031,2.047578,NaN,0.0,0.0,11.751012,0.0,0.183194,13.931912,914.763292,18.357752,69.776486,27.589406,9.089793,6.618346,NaN,NaN,NaN,2013-09-04,464213.322860,5.313488e+06
4,11.0,NaN,5.675208,1.893263,NaN,0.0,0.0,10.928064,0.0,0.139656,14.678689,909.438722,19.11439,70.604736,28.819854,9.356102,7.465027,NaN,NaN,NaN,2013-09-05,464213.322860,5.313488e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2430310,19172.0,NaN,9.317008,2.585573,1.0,3.2,4.0,0.0,0.0,7.632162,7.1,993.170075,2.6,96.38,4.7,1.2,1.0,eor,48.0,5986328.325147,2022-01-09,525418.165577,5.986328e+06
2430311,19172.0,NaN,4.326879,1.297722,1.0,0.0,0.0,0.016455,0.0,7.661153,7.1,1018.380426,2.7,95.54,4.0,1.7,0.6,eor,48.0,5986328.325147,2022-01-10,525418.165577,5.986328e+06
2430312,19172.0,NaN,7.767502,2.742065,1.0,0.5,4.0,1.885133,0.0,7.703287,6.9,1030.09153,2.7,92.29,4.1,1.5,1.7,eor,48.0,5986328.325147,2022-01-11,525418.165577,5.986328e+06
2430313,19172.0,NaN,9.417008,3.207821,1.0,0.0,4.0,0.706495,0.0,7.641972,8.2,1033.306002,4.3,98.42,6.2,2.3,2.3,eor,48.0,5986328.325147,2022-01-12,525418.165577,5.986328e+06


In [125]:
# dump data
df_interpolated.to_csv(f"{ROOT_PATH}/weather_data_interpolated.csv", index=False, encoding="utf-8")

In [5]:
df = pd.read_csv(f"{ROOT_PATH}/weather_data_interpolated.csv", encoding="utf-8")
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')

## Data Engineering
### Extreme Weather Conditions

In [6]:
df["heat_day"] = df["temp_max"].apply(lambda i: 1 if i > 30 else 0)
df["extreme_heat_day"] = df["temp_max"].apply(lambda i: 1 if i > 35 else 0)
df["heavy_rain"] = df["rain_mm"].apply(lambda i: 1 if i > 25 else 0)
df["storm"] = df["wind_max"].apply(lambda i: 1 if i > 20.8 else 0)
df["frozen_day"] = df["temp_mean"].apply(lambda i: 1 if i < 0 else 0)
df["high_snow"] = df["snow_cm"].apply(lambda i: 1 if i > 50 else 0)
df["ground_frost"] = df["temp_min_ground_level"].apply(lambda i: 1 if i < -5 else 0)

df.head()

,station_id,quality_level3,wind_max,wind_mean,quality_level4,rain_mm,rain_form_code,sun_h,snow_cm,cloud_cover_mean,vapor_pressure_mean,pressure_mean,temp_mean,humidity,temp_max,temp_min,temp_min_ground_level,eor,alt,station_name,date,x,y,heat_day,extreme_heat_day,heavy_rain,storm,frozen_day,high_snow,ground_frost
0,11.0,NaN,8.173478,2.038094,NaN,0.08561,0.0,4.654754,0.0,6.796674,12.210200,913.956126,14.419308,74.668488,19.478324,8.659381,7.848270,NaN,NaN,NaN,2013-09-01,464213.322860,5.313488e+06,0,0,0,0,0,0,0
1,11.0,NaN,9.092413,3.091648,NaN,0.00000,0.0,8.542860,0.0,3.454191,8.483979,915.470133,13.264470,61.811370,21.801163,6.271705,4.308915,NaN,NaN,NaN,2013-09-02,464213.322860,5.313488e+06,0,0,0,0,0,0,0
2,11.0,NaN,7.058365,2.097729,NaN,0.00000,0.0,11.991982,0.0,0.581327,11.820947,918.033514,16.294171,66.673953,26.441712,5.558288,4.209836,NaN,NaN,NaN,2013-09-03,464213.322860,5.313488e+06,0,0,0,0,0,0,0
3,11.0,NaN,6.929031,2.047578,NaN,0.00000,0.0,11.751012,0.0,0.183194,13.931912,914.763292,18.357752,69.776486,27.589406,9.089793,6.618346,NaN,NaN,NaN,2013-09-04,464213.322860,5.313488e+06,0,0,0,0,0,0,0
4,11.0,NaN,5.675208,1.893263,NaN,0.00000,0.0,10.928064,0.0,0.139656,14.678689,909.438722,19.114390,70.604736,28.819854,9.356102,7.465027,NaN,NaN,NaN,2013-09-05,464213.322860,5.313488e+06,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2430310,19172.0,NaN,9.317008,2.585573,1.0,3.20000,4.0,0.000000,0.0,7.632162,7.100000,993.170075,2.600000,96.380000,4.700000,1.200000,1.000000,eor,48.0,5.986328e+06,2022-01-09,525418.165577,5.986328e+06,0,0,0,0,0,0,0
2430311,19172.0,NaN,4.326879,1.297722,1.0,0.00000,0.0,0.016455,0.0,7.661153,7.100000,1018.380426,2.700000,95.540000,4.000000,1.700000,0.600000,eor,48.0,5.986328e+06,2022-01-10,525418.165577,5.986328e+06,0,0,0,0,0,0,0
2430312,19172.0,NaN,7.767502,2.742065,1.0,0.50000,4.0,1.885133,0.0,7.703287,6.900000,1030.091530,2.700000,92.290000,4.100000,1.500000,1.700000,eor,48.0,5.986328e+06,2022-01-11,525418.165577,5.986328e+06,0,0,0,0,0,0,0
2430313,19172.0,NaN,9.417008,3.207821,1.0,0.00000,4.0,0.706495,0.0,7.641972,8.200000,1033.306002,4.300000,98.420000,6.200000,2.300000,2.300000,eor,48.0,5.986328e+06,2022-01-12,525418.165577,5.986328e+06,0,0,0,0,0,0,0


In [7]:
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month

def apply_relevant_year(month: int, year: int)->int:
    if month >= 9:
        return year + 1
    else:
        return year
    
def apply_season(month: int)->str:
    if month >= 5 and month <= 8:
        return "summer"
    elif month >= 3 and month <= 4:
        return "spring"
    elif month >= 9 and month <= 11:
        return "autumn"
    else:
        return "winter"

df["relevant_year"] = df.apply(lambda x: apply_relevant_year(x["month"], x["year"]), axis=1)
df["season"] = df.apply(lambda x: apply_season(x["month"]), axis=1)

df.head()

,station_id,quality_level3,wind_max,wind_mean,quality_level4,rain_mm,rain_form_code,sun_h,snow_cm,cloud_cover_mean,vapor_pressure_mean,pressure_mean,temp_mean,humidity,temp_max,temp_min,temp_min_ground_level,eor,alt,station_name,date,x,y,heat_day,extreme_heat_day,heavy_rain,storm,frozen_day,high_snow,ground_frost,year,month,relevant_year,season
0,11.0,NaN,8.173478,2.038094,NaN,0.08561,0.0,4.654754,0.0,6.796674,12.210200,913.956126,14.419308,74.668488,19.478324,8.659381,7.848270,NaN,NaN,NaN,2013-09-01,464213.322860,5.313488e+06,0,0,0,0,0,0,0,2013,9,2014,autumn
1,11.0,NaN,9.092413,3.091648,NaN,0.00000,0.0,8.542860,0.0,3.454191,8.483979,915.470133,13.264470,61.811370,21.801163,6.271705,4.308915,NaN,NaN,NaN,2013-09-02,464213.322860,5.313488e+06,0,0,0,0,0,0,0,2013,9,2014,autumn
2,11.0,NaN,7.058365,2.097729,NaN,0.00000,0.0,11.991982,0.0,0.581327,11.820947,918.033514,16.294171,66.673953,26.441712,5.558288,4.209836,NaN,NaN,NaN,2013-09-03,464213.322860,5.313488e+06,0,0,0,0,0,0,0,2013,9,2014,autumn
3,11.0,NaN,6.929031,2.047578,NaN,0.00000,0.0,11.751012,0.0,0.183194,13.931912,914.763292,18.357752,69.776486,27.589406,9.089793,6.618346,NaN,NaN,NaN,2013-09-04,464213.322860,5.313488e+06,0,0,0,0,0,0,0,2013,9,2014,autumn
4,11.0,NaN,5.675208,1.893263,NaN,0.00000,0.0,10.928064,0.0,0.139656,14.678689,909.438722,19.114390,70.604736,28.819854,9.356102,7.465027,NaN,NaN,NaN,2013-09-05,464213.322860,5.313488e+06,0,0,0,0,0,0,0,2013,9,2014,autumn
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2430310,19172.0,NaN,9.317008,2.585573,1.0,3.20000,4.0,0.000000,0.0,7.632162,7.100000,993.170075,2.600000,96.380000,4.700000,1.200000,1.000000,eor,48.0,5.986328e+06,2022-01-09,525418.165577,5.986328e+06,0,0,0,0,0,0,0,2022,1,2022,winter
2430311,19172.0,NaN,4.326879,1.297722,1.0,0.00000,0.0,0.016455,0.0,7.661153,7.100000,1018.380426,2.700000,95.540000,4.000000,1.700000,0.600000,eor,48.0,5.986328e+06,2022-01-10,525418.165577,5.986328e+06,0,0,0,0,0,0,0,2022,1,2022,winter
2430312,19172.0,NaN,7.767502,2.742065,1.0,0.50000,4.0,1.885133,0.0,7.703287,6.900000,1030.091530,2.700000,92.290000,4.100000,1.500000,1.700000,eor,48.0,5.986328e+06,2022-01-11,525418.165577,5.986328e+06,0,0,0,0,0,0,0,2022,1,2022,winter
2430313,19172.0,NaN,9.417008,3.207821,1.0,0.00000,4.0,0.706495,0.0,7.641972,8.200000,1033.306002,4.300000,98.420000,6.200000,2.300000,2.300000,eor,48.0,5.986328e+06,2022-01-12,525418.165577,5.986328e+06,0,0,0,0,0,0,0,2022,1,2022,winter


In [92]:
def count_periods_without_rain(data_frame:pd.DataFrame, category:str="rain_mm", season_filter:str="summer", threshold:float=2.5, bound:float=8.0):
    result = dict()
    for station_id in data_frame.station_id.unique():
        result[station_id] = dict()
        
        for year in data_frame.relevant_year.unique():
            STREAK_COUNTER = 0.0
            DAYS_WITHOUT = 0.0
            
            sub_frame = data_frame.loc[(data_frame.station_id == station_id) & (data_frame.relevant_year == year) & (data_frame.season == season_filter) & (~data_frame[category].isna())]
            sub_frame = sub_frame.drop_duplicates()
            
            if sub_frame.shape[0] == 0:
                continue
                
            for _i, _row in sub_frame.iterrows():
                if _row[category] <= threshold: # no rain
                    DAYS_WITHOUT += 1.0
                    
                    if DAYS_WITHOUT == bound:
                        STREAK_COUNTER += 1
                else: # rain
                    DAYS_WITHOUT = 0.0
                    
            result[station_id][year] = STREAK_COUNTER
    return result

def get_longest_streak(data_frame:pd.DataFrame, category: str, threshold: float, operator: str="gt", season_filter: str="summer")->dict:
    """
    :param data_frame: panda's DataFrame to search
    :param category: category representing a pd.DataFrame column that should be looked at
    :param threshold: threshold that has to be met
    :param operator: operator to use for the threshold comparison (default = gt (greather than))
    :param season_fiter: to reduce data that has to be looped through (default = summer)
    :return: station dictionary with longest streak for each year
    """
    result = dict()
    for station_id in data_frame.station_id.unique():
        result[station_id] = dict()

        for year in data_frame.relevant_year.unique():
            STREAK_COUNTER = 0.0
            SMALL_COUNTER = 0.0

            sub_frame = data_frame.loc[(data_frame.station_id == station_id) & (data_frame.relevant_year == year) & (data_frame.season == season_filter) & (~data_frame[category].isna())]
            sub_frame = sub_frame.drop_duplicates()

            if sub_frame.shape[0] == 0:
                continue

            for _i, _row in sub_frame.iterrows():
                if operator == "gt":
                    if _row[category] >= threshold:
                        SMALL_COUNTER += 1.0
                    else:
                        # update overall counter
                        if SMALL_COUNTER > STREAK_COUNTER:
                            STREAK_COUNTER = SMALL_COUNTER
                        # reset counter
                        SMALL_COUNTER = 0.0
                else: # lt
                    if _row[category] <= threshold:
                        SMALL_COUNTER += 1.0
                    else:
                        # update overall counter
                        if SMALL_COUNTER > STREAK_COUNTER:
                            STREAK_COUNTER = SMALL_COUNTER
                        # reset counter
                        SMALL_COUNTER = 0.0

            result[station_id][year] = STREAK_COUNTER
    return result


In [ ]:
periods_without_rain_summer = count_periods_without_rain(data_frame=df, category="rain_mm", season_filter="summer", threshold=2.5, bound=9.0)
longest_heat_wave = get_longest_streak(data_frame=df, category="temp_max", operator='gt', season_filter="summer", threshold=27.5)
longest_cold_wave = get_longest_streak(data_frame=df, category="temp_max", operator='lt', season_filter="winter", threshold=0.5)

In [106]:
data = list()
for observation_id, values in longest_heat_wave.items():
    #print()
    for year, val in values.items():
        data.append([observation_id, year, val, "summer"])

df_longest_heat_wave = pd.DataFrame(data=data, columns=["station_id", "relevant_year", "longest_heat_wave", "season"])

data = list()
for observation_id, values in longest_cold_wave.items():
    #print()
    for year, val in values.items():
        data.append([observation_id, year, val, "winter"])

df_longest_cold_wave = pd.DataFrame(data=data, columns=["station_id", "relevant_year", "longest_cold_wave", "season"])

data = list()
for observation_id, values in periods_without_rain_summer.items():
    #print()
    for year, val in values.items():
        data.append([observation_id, year, val, "summer"])

df_periods_without_rain = pd.DataFrame(data=data, columns=["station_id", "relevant_year", "periods_without_rain_summer", "season"])

df = df.merge(df_periods_without_rain, on=["station_id", "relevant_year", "season"], how="left")
df["periods_without_rain_summer"] = df["periods_without_rain_summer"].fillna(0.0)

df = df.merge(df_longest_heat_wave, on=["station_id", "relevant_year", "season"], how="left")
df["longest_heat_wave"] = df["longest_heat_wave"].fillna(0.0)

df = df.merge(df_longest_cold_wave, on=["station_id", "relevant_year", "season"], how="left")
df["longest_cold_wave"] = df["longest_cold_wave"].fillna(0.0)

df.head()

,station_id,quality_level3,wind_max,wind_mean,quality_level4,rain_mm,rain_form_code,sun_h,snow_cm,cloud_cover_mean,vapor_pressure_mean,pressure_mean,temp_mean,humidity,temp_max,temp_min,temp_min_ground_level,eor,alt,station_name,date,x,y,heat_day,extreme_heat_day,heavy_rain,storm,frozen_day,high_snow,ground_frost,year,month,relevant_year,season,periods_without_rain_summer,periods_without_rain_spring,longest_heat_wave,longest_cold_wave
0,11.0,NaN,8.173478,2.038094,NaN,0.08561,0.0,4.654754,0.0,6.796674,12.210200,913.956126,14.419308,74.668488,19.478324,8.659381,7.848270,NaN,NaN,NaN,2013-09-01,464213.322860,5.313488e+06,0,0,0,0,0,0,0,2013,9,2014,autumn,0.0,0.0,0.0,0.0
1,11.0,NaN,9.092413,3.091648,NaN,0.00000,0.0,8.542860,0.0,3.454191,8.483979,915.470133,13.264470,61.811370,21.801163,6.271705,4.308915,NaN,NaN,NaN,2013-09-02,464213.322860,5.313488e+06,0,0,0,0,0,0,0,2013,9,2014,autumn,0.0,0.0,0.0,0.0
2,11.0,NaN,7.058365,2.097729,NaN,0.00000,0.0,11.991982,0.0,0.581327,11.820947,918.033514,16.294171,66.673953,26.441712,5.558288,4.209836,NaN,NaN,NaN,2013-09-03,464213.322860,5.313488e+06,0,0,0,0,0,0,0,2013,9,2014,autumn,0.0,0.0,0.0,0.0
3,11.0,NaN,6.929031,2.047578,NaN,0.00000,0.0,11.751012,0.0,0.183194,13.931912,914.763292,18.357752,69.776486,27.589406,9.089793,6.618346,NaN,NaN,NaN,2013-09-04,464213.322860,5.313488e+06,0,0,0,0,0,0,0,2013,9,2014,autumn,0.0,0.0,0.0,0.0
4,11.0,NaN,5.675208,1.893263,NaN,0.00000,0.0,10.928064,0.0,0.139656,14.678689,909.438722,19.114390,70.604736,28.819854,9.356102,7.465027,NaN,NaN,NaN,2013-09-05,464213.322860,5.313488e+06,0,0,0,0,0,0,0,2013,9,2014,autumn,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2430310,19172.0,NaN,9.317008,2.585573,1.0,3.20000,4.0,0.000000,0.0,7.632162,7.100000,993.170075,2.600000,96.380000,4.700000,1.200000,1.000000,eor,48.0,5.986328e+06,2022-01-09,525418.165577,5.986328e+06,0,0,0,0,0,0,0,2022,1,2022,winter,0.0,0.0,0.0,2.0
2430311,19172.0,NaN,4.326879,1.297722,1.0,0.00000,0.0,0.016455,0.0,7.661153,7.100000,1018.380426,2.700000,95.540000,4.000000,1.700000,0.600000,eor,48.0,5.986328e+06,2022-01-10,525418.165577,5.986328e+06,0,0,0,0,0,0,0,2022,1,2022,winter,0.0,0.0,0.0,2.0
2430312,19172.0,NaN,7.767502,2.742065,1.0,0.50000,4.0,1.885133,0.0,7.703287,6.900000,1030.091530,2.700000,92.290000,4.100000,1.500000,1.700000,eor,48.0,5.986328e+06,2022-01-11,525418.165577,5.986328e+06,0,0,0,0,0,0,0,2022,1,2022,winter,0.0,0.0,0.0,2.0
2430313,19172.0,NaN,9.417008,3.207821,1.0,0.00000,4.0,0.706495,0.0,7.641972,8.200000,1033.306002,4.300000,98.420000,6.200000,2.300000,2.300000,eor,48.0,5.986328e+06,2022-01-12,525418.165577,5.986328e+06,0,0,0,0,0,0,0,2022,1,2022,winter,0.0,0.0,0.0,2.0


In [127]:
df_date = df.copy()
df_date.index = pd.to_datetime(df_date['date'], format='%y-%m-%d')

df_date.drop(columns=["date"], inplace=True)
#df_date.index.month, df_date.index.year
df_mean_temp = df_date.groupby(by=["y", "x", "station_id", "season", "relevant_year"]).agg({"temp_mean": "mean", "sun_h": "sum", "vapor_pressure_mean": "mean", "humidity": "mean", "rain_mm": "sum", "wind_mean": "mean", "snow_cm": "sum", "pressure_mean": "mean", "periods_without_rain_summer": "mean", "periods_without_rain_spring": "mean", "longest_heat_wave": "mean", "longest_cold_wave": "mean"})
#df_mean_temp.index.rename(names=["x", "y", "station_id", "month", "year"], inplace=True)
df_mean_temp = df_mean_temp.reset_index()

df_mean_temp.head()

,y,x,station_id,season,relevant_year,temp_mean,sun_h,vapor_pressure_mean,humidity,rain_mm,wind_mean,snow_cm,pressure_mean,periods_without_rain_summer,periods_without_rain_spring,longest_heat_wave,longest_cold_wave
0,5.250228e+06,596276.982744,3730.0,autumn,2014,7.709890,283.4000,9.624176,87.956044,485.40,1.871429,96.0,923.990110,0.0,0.0,0.0,0.0
1,5.250228e+06,596276.982744,3730.0,autumn,2015,9.175824,362.8500,10.119780,85.868132,319.95,1.701648,3.0,925.250549,0.0,0.0,0.0,0.0
2,5.250228e+06,596276.982744,3730.0,autumn,2016,7.740659,335.3000,9.138462,84.752747,263.80,1.960440,50.0,937.164835,0.0,0.0,0.0,0.0
3,5.250228e+06,596276.982744,3730.0,autumn,2017,8.902747,304.4500,9.969780,85.021978,147.60,2.374725,2.0,966.817033,0.0,0.0,0.0,0.0
4,5.250228e+06,596276.982744,3730.0,autumn,2018,7.289011,306.4300,8.917582,85.338736,413.10,2.091209,64.0,939.311648,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26787,6.096173e+06,462428.994785,3032.0,winter,2018,2.310833,318.0460,6.422500,86.394528,274.70,5.699722,1.5,1000.600583,0.0,0.0,0.0,4.0
26788,6.096173e+06,462428.994785,3032.0,winter,2019,3.574444,351.1675,6.794722,84.993278,259.65,5.817500,6.0,1004.106528,0.0,0.0,0.0,6.0
26789,6.096173e+06,462428.994785,3032.0,winter,2020,5.089560,273.1945,7.458242,84.672940,360.40,6.384890,0.0,1000.155714,0.0,0.0,0.0,0.0
26790,6.096173e+06,462428.994785,3032.0,winter,2021,2.254444,294.8200,6.455278,87.638167,214.20,4.356667,107.0,999.482639,0.0,0.0,0.0,8.0


### Seasonal Weather Records

In [129]:
column_season_combinations = dict(
    sun_h = ["spring", "summer", "autumn", "winter"],
    humidity = ["spring", "summer", "autumn", "winter"],
    rain_mm = ["spring", "summer", "autumn", "winter"],
    snow_cm = ["winter"],
    periods_without_rain_summer = ["summer"],
    periods_without_rain_spring = ["spring"],
    longest_heat_wave = ["summer"],
    longest_cold_wave = ["winter"],
)

monthly_weather_data = df.groupby(["x", "y", "relevant_year", "month"]).agg({"heat_day": "sum", "extreme_heat_day": "sum", "heavy_rain": "sum", "storm": "sum", "frozen_day": "sum", "temp_mean": "mean", "sun_h": "sum", "vapor_pressure_mean": "mean", "humidity": "mean", "rain_mm": "sum", "wind_mean": "mean", "snow_cm": "mean", "pressure_mean": "mean"}).reset_index()
monthly_weather_data['date'] = monthly_weather_data.apply(lambda row: datetime.strptime(f"{int(row.relevant_year)}-{int(row.month)}-15", '%Y-%m-%d'), axis=1)

season_weather_data =  df.groupby(["x", "y", "relevant_year", "season"]).agg({"heat_day": "sum", "extreme_heat_day": "sum", "heavy_rain": "sum", "storm": "sum", "frozen_day": "sum", "temp_mean": "mean", "sun_h": "sum", "vapor_pressure_mean": "mean", "humidity": "mean", "rain_mm": "sum", "wind_mean": "mean", "snow_cm": "mean", "pressure_mean": "mean"}).reset_index()
season_weather_data.rename(columns={"month_temp_mean": "season_temp_mean", "month_temp_max": "season_temp_max"}, inplace=True)

yearly_weather_data = df.groupby(["x", "y", "relevant_year"]).agg({"heat_day": "sum", "extreme_heat_day": "sum", "heavy_rain": "sum", "storm": "sum", "frozen_day": "sum"}).reset_index()
yearly_weather_data.rename(columns={"relevant_year": "date"}, inplace=True)

for _i, row in yearly_weather_data.iterrows():
    seasons = df_mean_temp.loc[(df_mean_temp["relevant_year"] == row["date"]) & (df_mean_temp["x"] == row["x"]) & (df_mean_temp["y"] == row["y"])] #[["season", "temp_mean", "sun_h", "vapor_pressure_mean", "humidity", "rain_mm", "wind_mean", "snow_cm", "pressure_mean", "periods_without_rain_summer", "periods_without_rain_spring", "longest_heat_wave", "longest_cold_wave"]]
        
    for _j, season_row in seasons.iterrows():
        for col in seasons.columns:
            if col in column_season_combinations and season_row['season'] in column_season_combinations[col]:
                yearly_weather_data.at[_i, f"{season_row['season']}_{col}"] = season_row[col]
        
yearly_weather_data.head()

,x,y,date,heat_day,extreme_heat_day,heavy_rain,storm,frozen_day,autumn_sun_h,autumn_humidity,autumn_rain_mm,spring_sun_h,spring_humidity,spring_rain_mm,spring_periods_without_rain_spring,summer_sun_h,summer_humidity,summer_rain_mm,summer_periods_without_rain_summer,summer_longest_heat_wave,winter_sun_h,winter_humidity,winter_rain_mm,winter_snow_cm,winter_longest_cold_wave
0,290318.264908,5.631617e+06,2014,4,0,6,20,0,274.2000,85.000000,235.00,354.900,71.016393,26.7,0.0,728.500,76.089431,471.00,5.0,3.0,244.900,80.500000,118.90,0.0,0.0
1,290318.264908,5.631617e+06,2015,13,0,1,16,13,364.6000,83.164835,173.70,343.800,76.180328,94.7,0.0,902.200,73.195122,236.05,4.0,5.0,164.600,89.688889,207.10,13.0,2.0
2,290318.264908,5.631617e+06,2016,7,0,1,17,7,301.1500,87.796703,206.80,248.600,77.000000,133.2,0.0,772.550,75.365854,253.25,5.0,6.0,227.200,83.835165,255.20,15.0,3.0
3,290318.264908,5.631617e+06,2017,6,0,0,5,29,364.2500,81.505495,113.90,314.650,72.445738,70.7,0.0,835.828,72.915854,247.20,6.0,4.0,229.600,85.827778,115.55,116.0,5.0
4,290318.264908,5.631617e+06,2018,13,0,2,15,24,283.3840,84.913132,189.75,269.586,75.042295,150.4,0.0,1025.696,68.579593,166.55,4.0,13.0,173.918,86.660556,203.70,18.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7087,915934.426750,5.684702e+06,2018,24,1,0,6,40,262.0110,84.405714,185.75,401.400,72.415082,73.8,0.0,1083.367,63.928943,99.20,8.0,20.0,209.367,82.944111,91.20,36.0,2.0
7088,915934.426750,5.684702e+06,2019,16,2,1,4,31,472.9340,73.119341,98.60,368.085,70.037377,62.1,0.0,997.096,68.761463,178.25,6.0,7.0,200.933,82.674778,165.00,111.0,6.0
7089,915934.426750,5.684702e+06,2020,8,0,2,6,7,368.6915,79.950824,121.20,482.483,61.170000,37.4,0.0,911.353,69.576829,238.40,5.0,3.0,202.367,80.959451,152.20,0.0,0.0
7090,915934.426750,5.684702e+06,2021,4,0,3,3,35,366.5350,81.430165,157.00,290.936,71.277869,69.3,0.0,844.377,74.436341,266.95,6.0,4.0,220.934,83.257889,109.50,155.0,10.0


In [135]:
yearly_weather_data.describe()

,x,y,date,heat_day,extreme_heat_day,heavy_rain,storm,frozen_day,autumn_sun_h,autumn_humidity,autumn_rain_mm,spring_sun_h,spring_humidity,spring_rain_mm,summer_sun_h,summer_humidity,summer_rain_mm,summer_periods_without_rain_summer,summer_longest_heat_wave,winter_sun_h,winter_humidity,winter_rain_mm,winter_snow_cm,winter_longest_cold_wave
count,7092.000000,7.092000e+03,7092.000000,7092.000000,7092.000000,7092.000000,7092.000000,7092.000000,7092.000000,7092.000000,7092.000000,6304.000000,6304.000000,6304.000000,6304.000000,6304.000000,6304.000000,6304.000000,6304.000000,7092.000000,7092.000000,7092.000000,7092.000000,7092.000000
mean,586078.056840,5.642791e+06,2018.000000,9.481669,0.874647,2.096447,7.564439,25.107304,327.654654,83.989842,171.195715,347.675811,72.802723,86.743861,896.178295,71.912147,297.355584,9.069956,6.583756,167.680867,86.099260,167.358806,204.492821,4.788635
std,145557.097884,2.172773e+05,2.582171,8.266184,1.686866,2.265486,11.418140,19.620829,71.684564,3.547897,76.226750,72.506459,5.284912,38.538551,135.968835,4.788468,123.601992,5.462406,3.992293,60.339652,3.516432,86.701233,911.143881,4.047806
min,290318.264908,5.250228e+06,2014.000000,0.000000,0.000000,0.000000,0.000000,0.000000,147.015000,68.847363,36.800000,191.100000,55.546501,19.500000,548.600000,55.550325,32.450000,1.000000,0.000000,23.550731,57.755556,23.300000,0.000000,0.000000
25%,471975.040480,5.470573e+06,2016.000000,3.000000,0.000000,1.000000,3.000000,12.000000,279.344558,81.879262,117.648642,301.451813,69.602704,60.712275,815.121425,68.667138,208.690532,5.000000,4.000000,140.843228,84.024724,108.768944,9.000000,2.000000
50%,571872.071061,5.636385e+06,2018.000000,8.000000,0.000000,2.000000,5.000000,20.000000,319.090293,84.212790,157.919908,342.744922,73.074475,80.500000,881.862384,72.342280,283.514036,7.000000,6.000000,169.705284,86.293357,150.762059,36.318901,4.000000
75%,709821.596912,5.811242e+06,2020.000000,14.000000,1.000000,3.000000,8.000000,34.000000,359.847696,86.298274,208.248895,374.997418,76.262295,104.473407,960.625892,75.571616,361.091142,12.000000,8.000000,198.959110,88.369471,207.200000,130.494094,7.000000
max,915934.426750,6.096173e+06,2022.000000,74.000000,18.000000,30.000000,151.000000,262.000000,1061.765794,95.340659,790.403984,1013.489269,89.672131,457.300000,2216.126000,89.943089,1113.174912,32.000000,28.000000,659.034746,98.222222,993.344444,28613.000000,77.000000


In [132]:
season_pivot = season_weather_data.pivot(index=["x", "y", "relevant_year"], columns="season", values=["temp_mean", "sun_h", "vapor_pressure_mean", "humidity", "rain_mm", "wind_mean", "snow_cm", "pressure_mean"]).reset_index()
new_cols = [col[0] for col in season_pivot.columns[:3]]
new_cols += [col[0].replace("season", col[1]) for col in season_pivot.columns[3:]]
season_pivot.columns=new_cols

## Tabular Data to Raster Format
### Linear Interpolation for Missing Values

In [137]:
# reset index of dataframe
monthly_weather_data_reindex = monthly_weather_data.set_index(["x", "y", "date"])
yearly_weather_data_reindex = yearly_weather_data.set_index(["x", "y", "date"])

In [ ]:
# non Dask function
def interpolate_missing_data(data: np.ndarray, mask: np.ndarray, method: str='nearest', fill_value: int=0):
    """
    :param data: a 2D image
    :param mask: a 2D boolean image, True indicates missing values
    :param method: interpolation method, one of
        'nearest', 'linear', 'cubic'.
    :param fill_value: which value to use for filling up data outside the
        convex hull of known pixel values.
        Default is 0, Has no effect for 'nearest'.
    :return: the image with missing values interpolated
    """

    h, w = data.shape[:2]
    xx, yy = np.meshgrid(np.arange(w), np.arange(h))

    known_x = xx[~mask]
    known_y = yy[~mask]
    known_v = data[~mask]
    missing_x = xx[mask]
    missing_y = yy[mask]

    interp_values = interpolate.griddata(
        (known_x, known_y), known_v, (missing_x, missing_y),
        method=method, fill_value=fill_value
    )

    interp_data = data.copy()
    interp_data[missing_y, missing_x] = interp_values

    return interp_data

In [ ]:
# use panda's xarray integration to transform DataFrame to xarray's Dataset
ds_year = yearly_weather_data_reindex.to_xarray()

for var in ds_year:
    interpolated_data = list()
    for _index_year, year in enumerate(ds_year["date"].values):
        try:
            data = ds_year[var].sel({"date": year}).values
            interp_data_linear = interpolate_missing_data(data=np.array(data).T, mask=np.isnan(np.array(data).T), method="linear", fill_value=np.NaN)
            interp_data_nearest = interpolate_missing_data(data=interp_data_linear, mask=np.isnan(interp_data_linear), method="nearest", fill_value=np.nanmin(interp_data_linear)).astype(int).astype(float)
            interpolated_data.append(interp_data_nearest)
        except (ValueError):
            interpolated_data.append(np.array(data).T)

    ds_year.update({var:(('x', 'y', 'date'), np.array(interpolated_data).T)})

ds_year

In [140]:
ds_year.to_netcdf(path=f"{ROOT_PATH}/historic_weather_data_year.nc", engine="netcdf4")

In [108]:
import geopandas as gpd

gdf_germany = gpd.read_file("./data/germany_shape/gadm36_DEU_0.shp")
gdf_germany = gdf_germany.explode("geometry").reset_index().drop(columns=["level_0", "level_1"])
gdf_germany["area"] = gdf_germany["geometry"].apply(lambda p: p.area)
gdf_germany = gdf_germany.sort_values(by="area", ascending=False)
gdf_germany = gdf_germany.to_crs("EPSG:32632")

gdf_germany.iloc[0].geometry

In [117]:
ds_year = ds_year.rio.write_crs(32632)
ds_year = ds_year.rio.clip(gdf_germany.geometry, crs=gdf_germany.crs)

In [119]:
# use panda's xarray integration to transform DataFrame to xarray's Dataset
ds_month = monthly_weather_data_reindex.to_xarray()

# fill missing values between weather stations
for var in ds_month:
    interpolated_data = list()
    for _index_date, date in enumerate(ds_month["date"].values):
        try:
            data = ds_month[var].sel({"date": date}).values
            interp_data_linear = interpolate_missing_data(data=np.array(data).T, mask=np.isnan(np.array(data).T), method="linear", fill_value=np.NaN)
            interp_data_nearest = interpolate_missing_data(data=interp_data_linear, mask=np.isnan(interp_data_linear), method="nearest", fill_value=np.nanmin(interp_data_linear)).astype(int).astype(float)
            interpolated_data.append(interp_data_nearest)
        except:
            interpolated_data.append(np.array(data).T)
            
    ds_month.update({var:(('y', 'x', 'date'), np.array(interpolated_data).T)})

ds_month

<xarray.Dataset>
Dimensions:           (x: 628, y: 628, date: 88)
Coordinates:
  * x                 (x) float64 2.903e+05 2.97e+05 ... 9.039e+05 9.159e+05
  * y                 (y) float64 5.25e+06 5.254e+06 ... 6.076e+06 6.096e+06
  * date              (date) datetime64[ns] 2015-01-15 2015-02-15 ... 2022-12-15
Data variables:
    relevant_year     (y, x, date) float64 2.015e+03 2.015e+03 ... 2.022e+03
    month             (y, x, date) float64 1.0 2.0 3.0 4.0 ... 10.0 11.0 12.0
    heat_day          (y, x, date) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    extreme_heat_day  (y, x, date) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    heavy_rain        (y, x, date) float64 0.0 0.0 0.0 0.0 ... 1.0 0.0 1.0 0.0
    storm             (y, x, date) float64 0.0 0.0 2.0 0.0 ... 1.0 3.0 4.0 3.0
    frozen_day        (y, x, date) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 10.0
    temp_mean         (y, x, date) float64 3.0 1.0 6.0 10.0 ... 11.0 7.0 2.0
    temp_max          (y, x, date) float64 7.0 5.0 12.0 17.0 ... 13.0 8.0 3.0

In [120]:
ds_month = ds_month.rio.write_crs(32632)
ds_month = ds_month.rio.clip(gdf_germany.geometry, crs=gdf_germany.crs)

In [274]:
ds_month.to_netcdf(path=f"{ROOT_PATH}/historic_weather_data_month.nc", engine="netcdf4")

## Weather Stations - Geo Locations

In [43]:
df_ws_locations = df_weather_stations.loc[df_weather_stations["von_datum"] > '2010-06-01'].groupby("station_id").agg({"lat": "mean", "long": "mean"}).reset_index()

In [82]:
gdf_ws_locations = gpd.GeoDataFrame(df_ws_locations, geometry=gpd.points_from_xy(df_ws_locations.long, df_ws_locations.lat))

In [74]:
gdf_ws_locations["geometry"] = gdf_ws_locations["geometry"].buffer(0.0025)
gdf_ws_locations

,station_id,lat,long,geometry
0,96,52.943700,12.851800,"POLYGON ((12.86680 52.94370, 12.86673 52.94223..."
1,125,47.834200,10.866700,"POLYGON ((10.88170 47.83420, 10.88163 47.83273..."
2,161,50.423700,7.420200,"POLYGON ((7.43520 50.42370, 7.43513 50.42223, ..."
3,164,53.031600,13.990800,"POLYGON ((14.00580 53.03160, 14.00573 53.03013..."
4,167,53.841067,13.684967,"POLYGON ((13.69997 53.84107, 13.69989 53.83960..."
...,...,...,...,...
163,15911,51.134800,14.434300,"POLYGON ((14.44930 51.13480, 14.44923 51.13333..."
164,15976,49.952900,8.210700,"POLYGON ((8.22570 49.95290, 8.22563 49.95143, ..."
165,15978,49.504400,7.861800,"POLYGON ((7.87680 49.50440, 7.87673 49.50293, ..."
166,19171,54.003800,9.855300,"POLYGON ((9.87030 54.00380, 9.87023 54.00233, ..."


In [93]:
geodf = gpd.read_file(f"./data/ThuenenGeoLocations/geolocations_aoi.geojson").to_crs("EPSG:4326")
geodf = geodf.loc[geodf.bez_wg_bu == "Schwarzwald"]
geodf

,gid,bez_wg_bu,geometry
5,73,Schwarzwald,"POLYGON ((8.25331 47.77714, 8.25504 47.77668, ..."


In [ ]:
import ipyleaflet
from ipywidgets import Layout
from ipyleaflet import basemaps, Polygon

defaultLayout=Layout(width='400px', height='500px')

m = ipyleaflet.Map(basemap=basemaps.CartoDB.Positron, scroll_wheel_zoom=False, center=(geodf.iloc[0].geometry.centroid.y, geodf.iloc[0].geometry.centroid.x), zoom=8., layout=defaultLayout)

# add controls
m.add_control(ipyleaflet.ScaleControl(position="bottomright"))
m.add_control(ipyleaflet.FullScreenControl())

bf_loc = list()
for t in list(geodf.iloc[0].geometry.exterior.coords):
    bf_loc.append((t[1], t[0]))

polygon_bf = Polygon(
    locations=bf_loc,
    color="green",
    weight=3,
    fill=True
)

m.add_layer(polygon_bf)

for i, row in gdf_ws_locations.iterrows():
    location = list()
    #for t in list(row.geometry.exterior.coords):
    #    location.append((t[1], t[0]))

    polygon_germany = Polygon(
        locations=location,
        color="purple",
        weight=3,
        fill=False
    )
    #m.add_layer(polygon_germany)
    icon = ipyleaflet.AwesomeIcon(
        name='thermometer-empty',
        marker_color='blue',
        icon_color='white',
        spin=False
    )
    marker = ipyleaflet.Marker(icon=icon, location=(row.geometry.y, row.geometry.x), draggable=False)
    m.add_layer(marker)

m

In [ ]:
df_ws_locations.hvplot.points(x='long', y='lat', geo=True, color='red', alpha=1, tiles='OSM', title='DWD Wetter Stationen', hover_cols='station_id', width=400, height=600, xlabel="longitude", ylabel="latitude")